In [1]:
import pandas as pd
import importlib
import fileDistanceProcessing as fdp
import importlib
importlib.reload(fdp) 

<module 'fileDistanceProcessing' from '/home/marko/Desktop/ip2/ip2_projekat/source/fileDistanceProcessing.py'>

In [2]:
X=pd.read_csv("../data/csv/sars2_mers_sars1.txt",skipinitialspace = True )

In [3]:
X.columns = X.columns.str.strip()
X.columns

Index(['Virus', 'ACCESSION', 'WHO_TYPE', 'ACCESSION_PROT', 'PROTEIN',
       'DUZINA_NUC', 'KOD_NUC'],
      dtype='object')

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [5]:
distances = fdp.deserialize_and_decompress_distance_matrix("../data/full_distances_matrix.7z")

In [6]:
# metrika zasnovana samo na 5-adic distanci
# instant racunanje iz matrice
def five_adic_distance_from_matrix(x, y):
    
    i = int(x[-1]) #indeks je na poslednjem mestu u listi
    j = int(y[-1])

    # matrica je u obliku trougla bez dijagonale originalne matrice
    first = min(i, j)
    second = max(i, j) - (first + 1) # veci indeks smanji za broj instanci u redu matrice do dijagonale 

    return 0 if i == j else distances[first][second] #udaljenost x od y

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

def visualize_confusion_matrix(cm, labels, save_path=""):
    """
    Funkcija za prikaz matrice konfuzije sa bojama:
    - Dijagonala: Zelena
    - Greške: Crvene
    - Ostalo: Belo

    Parametri:
    cm : Matrica konfuzije.
    labels : Lista labela (naziva klasa).
    save_path : Putanja za čuvanje slike. Ako se izostavi slika se ne čuva.
    """
    
    mask_diagonal = np.eye(len(cm), dtype=bool)
    mask_errors = ~mask_diagonal

    # Postavljanje boja
    cmap = sns.color_palette(["white"])  
    cmap_diagonal = sns.color_palette(["lightgreen"])  
    cmap_errors = sns.color_palette(["red"])  

    colors = np.array([[cmap[0]] * len(cm)] * len(cm))

    # Postavljanje boja za dijagonalu
    colors[mask_diagonal] = cmap_diagonal[0]

    # Postavljanje boja za greške
    colors[mask_errors & (cm > 0)] = cmap_errors[0]

    # Vizualizacija
    plt.figure(figsize=(11.69, 8.27), dpi=600)  # A4 format (11.69x8.27 inča)
    ax = sns.heatmap(
        cm, 
        annot=True,  # Ispis vrednosti u ćelijama
        fmt='d',  # Formatiranje kao celi brojevi
        cmap=cmap,  # Osnovna boja (belo)
        xticklabels=labels, 
        yticklabels=labels, 
        mask=False,  # Ne maskiramo nule
        cbar=False,  # Bez colorbar-a
        annot_kws={"size": 10, "color": "black"},  # Povećan font za brojeve i crna boja teksta
        linewidths=0,  # Bez linija između ćelija (ručno ćemo ih dodati)
        square=True  # Čini ćelije kvadratnim
    )

    # dodavanje linija između ćelija
    for i in range(len(cm) + 1):
        ax.axhline(i, color='black', linewidth=0.5)  # Horizontalne linije
        ax.axvline(i, color='black', linewidth=0.5)  # Vertikalne linije

    plt.gca().add_collection(plt.pcolormesh(np.arange(len(cm) + 1), np.arange(len(cm) + 1), colors, shading='flat'))
    plt.xlabel('Predviđene klase', fontsize=14)  # Povećan font za oznake
    plt.ylabel('Stvarne klase', fontsize=14)  # Povećan font za oznake
    plt.xticks(fontsize=12)  # Povećan font za x-ose
    plt.yticks(fontsize=12)  # Povećan font za y-ose

    # Čuvanje slike ako je putanja navedena
    if save_path!="": #and save_path!="train_" and save_path!="test_" :
        plt.savefig("../results/heatmaps/cross_validation/"+save_path+".png", bbox_inches='tight', dpi=600)  # Visoka rezolucija
    #plt.show()
    plt.close()

In [8]:
from sklearn.metrics import make_scorer
from sklearn.metrics import f1_score

In [9]:
k = 1
fold = 1
num_folds = 3
data = 'test'
classification = ['virus', 'protein', 'who']
index = 0
def custom_f1_loss(y_true, y_pred):
    global k, fold, data
    cm=[]
    labels = y_true.unique()
    cm= confusion_matrix(y_true,y_pred,labels=labels)

    visualize_confusion_matrix(cm, labels=labels, save_path=f'{classification[index]} {data}_k={k}_fold={fold}')
    
    if fold == num_folds:
        if data == 'train':
            fold = 1
            k += 1
            data = 'test'
        else:
            data = 'train'
    else:
        if data == 'train':
            fold += 1
            data = 'test'
        else:
            data = 'train'

    return f1_score(y_true, y_pred, labels=labels, average='macro')

In [10]:
custom_f1 = make_scorer(custom_f1_loss, greater_is_better=True)

In [11]:
y_virus=X["Virus"]
X_virus=[] # potreban nam je samo indeks

# na svaku instancu X_virus smo appendovali njen index
for i in range(len(y_virus)):
    X_virus.append([0, i]) # 0 je dodata zbog knn da ne pravi problem

In [12]:
from sklearn.model_selection import GridSearchCV

In [13]:
param_grid = {'n_neighbors': range(4, 6),
              'metric': [five_adic_distance_from_matrix]}

In [14]:
from sklearn.neighbors import KNeighborsClassifier

In [15]:
gs = GridSearchCV(KNeighborsClassifier(),
             param_grid=param_grid,
             scoring=custom_f1,
             cv=num_folds,
             verbose=5, return_train_score=True)

In [21]:
gs.fit(X_virus, y_virus)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=4;, score=(train=0.912, test=0.663) total time=  52.6s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=4;, score=(train=0.888, test=0.760) total time=  57.7s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=4;, score=(train=0.783, test=0.665) total time=  59.2s
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=5;, score=(train=0.862, test=0.663) total time=  47.1s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=5;, score=(train=0.832, test=0.760) total time=  50.7s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7fc2feb90c10>, n_neighbors=5;, score=(train=0.665, test=0.663) total time=  56.4s


GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'metric': [<function five_adic_distance_from_matrix at 0x7fc2feb90c10>],
                         'n_neighbors': range(4, 6)},
             return_train_score=True, scoring=make_scorer(custom_f1_loss),
             verbose=5)

In [16]:
k = 1
fold = 1
num_folds = 3
data = 'test'
index = 1

In [17]:
y_protein = X["PROTEIN"]
X_protein = X_virus

In [18]:
gs.fit(X_protein, y_protein)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/home/marko/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=1;, score=(train=0.997, test=0.960) total time=  49.7s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=1;, score=(train=0.999, test=0.987) total time=  52.4s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=1;, score=(train=0.999, test=0.972) total time= 1.0min
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=2;, score=(train=0.980, test=0.950) total time=  47.6s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=2;, score=(train=0.992, test=0.982) total time=  52.4s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=2;, score=(train=0.974, test=0.965) total time= 1.0min
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7fe59f442160>, n_neighbors=3;, score=(train=0.

KeyboardInterrupt: 

In [16]:
import numpy as np

virusTypes = np.array(X['Virus'])
X_who = []
indexes = []
for i in range(len(virusTypes)):
    if(virusTypes[i].strip() == "SARS_COV_2"):
        X_who.append([0, i])
        indexes.append(i)

sars2_only = X.filter(items=indexes, axis=0)

In [17]:
y_who=sars2_only["WHO_TYPE"].str.strip()

In [18]:
k = 4
fold = 1
num_folds = 3
data = 'test'
index = 2

In [19]:
gs.fit(X_who, y_who)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=4;, score=(train=0.961, test=0.882) total time=  38.6s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=4;, score=(train=0.949, test=0.947) total time=  39.5s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=4;, score=(train=0.955, test=0.950) total time=  39.5s
[CV 1/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=5;, score=(train=0.960, test=0.882) total time=  38.3s
[CV 2/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=5;, score=(train=0.944, test=0.946) total time=  38.8s
[CV 3/3] END metric=<function five_adic_distance_from_matrix at 0x7f712cf86430>, n_neighbors=5;, score=(train=0.957, test=0.943) total time=  40.6s


GridSearchCV(cv=3, estimator=KNeighborsClassifier(),
             param_grid={'metric': [<function five_adic_distance_from_matrix at 0x7f712cf86430>],
                         'n_neighbors': range(4, 6)},
             return_train_score=True, scoring=make_scorer(custom_f1_loss),
             verbose=5)